## 환경설정

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/BOAZ/[202207]문화공모전

/content/drive/MyDrive/BOAZ/[202207]문화공모전


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# 시각화 관련
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from collections import Counter
from PIL import Image

In [5]:
import matplotlib as mpl
mpl.rcParams['axes.unicode_minus'] = False

# 그래프에서 한글 폰트 깨지는 문제에 대한 대처(전역 글꼴 설정)
!apt-get update -qq
!apt-get install fonts-nanum* -qq

import matplotlib.font_manager as fm
import warnings
warnings.filterwarnings(action='ignore') 

path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf' # '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
font_name = fm.FontProperties(fname=path, size=10).get_name() # 기본 폰트 사이즈 : 10


plt.rc('font', family='NanumGothic')
fm._rebuild()

Selecting previously unselected package fonts-nanum.
(Reading database ... 155676 files and directories currently installed.)
Preparing to unpack .../fonts-nanum_20170925-1_all.deb ...
Unpacking fonts-nanum (20170925-1) ...
Selecting previously unselected package fonts-nanum-eco.
Preparing to unpack .../fonts-nanum-eco_1.000-6_all.deb ...
Unpacking fonts-nanum-eco (1.000-6) ...
Selecting previously unselected package fonts-nanum-extra.
Preparing to unpack .../fonts-nanum-extra_20170925-1_all.deb ...
Unpacking fonts-nanum-extra (20170925-1) ...
Selecting previously unselected package fonts-nanum-coding.
Preparing to unpack .../fonts-nanum-coding_2.5-1_all.deb ...
Unpacking fonts-nanum-coding (2.5-1) ...
Setting up fonts-nanum-extra (20170925-1) ...
Setting up fonts-nanum (20170925-1) ...
Setting up fonts-nanum-coding (2.5-1) ...
Setting up fonts-nanum-eco (1.000-6) ...
Processing triggers for fontconfig (2.12.6-0ubuntu2) ...


## 파일 불러오기

In [ ]:
data = pd.read_csv('코로나19이전이후문화시설유동인구.csv')
data.head()

,sn,fclty_id,fclty_nm,fclty_la,fclty_lo,prvyy_base_de,thsyr_base_de,prvyy_base_lcls_total_co,prvyy_base_otsd_total_co,prvyy_base_frnr_total_co,thsyr_base_lcls_total_co,thsyr_base_otsd_total_co,thsyr_base_frnr_total_co,thsyr_base_lcls_irds_co,thsyr_base_otsd_irds_co,thsyr_base_frnr_irds_co,thsyr_base_lcls_irds_rt,thsyr_base_otsd_irds_rt,thsyr_base_frnr_irds_rt
0,1,KP0001,396커피 컴퍼니,36.560047,128.699928,20190101,20200101,1326,453,0,1445.0,276.0,0.0,119.0,-177.0,0.0,8.97,-39.07,0.0
1,2,KP0001,396커피 컴퍼니,36.560047,128.699928,20190102,20200102,1534,286,0,1562.0,316.0,0.0,28.0,30.0,0.0,1.83,10.49,0.0
2,3,KP0001,396커피 컴퍼니,36.560047,128.699928,20190104,20200104,1571,335,0,1737.0,621.0,0.0,166.0,286.0,0.0,10.57,85.37,0.0
3,4,KP0001,396커피 컴퍼니,36.560047,128.699928,20190106,20200106,1433,445,0,1639.0,299.0,0.0,206.0,-146.0,0.0,14.38,-32.81,0.0
4,5,KP0001,396커피 컴퍼니,36.560047,128.699928,20190107,20200107,1517,347,0,1848.0,319.0,0.0,331.0,-28.0,0.0,21.82,-8.07,0.0


In [ ]:
data.info()
#데이터가..아주많다!ㅋㅋㅋ

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 992554 entries, 0 to 992553
Data columns (total 19 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   sn                        992554 non-null  int64  
 1   fclty_id                  992554 non-null  object 
 2   fclty_nm                  992554 non-null  object 
 3   fclty_la                  992005 non-null  float64
 4   fclty_lo                  992005 non-null  float64
 5   prvyy_base_de             992554 non-null  int64  
 6   thsyr_base_de             992554 non-null  int64  
 7   prvyy_base_lcls_total_co  992554 non-null  int64  
 8   prvyy_base_otsd_total_co  992554 non-null  int64  
 9   prvyy_base_frnr_total_co  992554 non-null  int64  
 10  thsyr_base_lcls_total_co  992554 non-null  float64
 11  thsyr_base_otsd_total_co  992554 non-null  float64
 12  thsyr_base_frnr_total_co  992554 non-null  float64
 13  thsyr_base_lcls_irds_co   992554 non-null  f

In [ ]:
#칼럼이름 알기쉽게 변경_그래프를 그리기에도 편리
data.rename(columns ={'sn':'일련번호',
                      'fclty_id':'시설ID',
                      'fclty_nm':'시설명',
                      'fclty_la':'위도',
                      'fclty_lo':'경도',
                      'prvyy_base_de':'전년기준일자',
                      'thsyr_base_de':'금년기준일자',
                      'prvyy_base_lcls_total_co':'전년현지인총합',
                      'prvyy_base_otsd_total_co':'전년외지인총합',
                      'prvyy_base_frnr_total_co':'전년외국인총합',
                      'thsyr_base_lcls_total_co':'금년현지인총합',
                      'thsyr_base_otsd_total_co':'금년외지인총합',
                      'thsyr_base_frnr_total_co':'금년외국인총합',
                      'thsyr_base_lcls_irds_co':'금년현지인증감수',
                      'thsyr_base_otsd_irds_co':'금년외지인증감수',
                      'thsyr_base_frnr_irds_co':'금년외국인증감수',
                      'thsyr_base_lcls_irds_rt':'금년현지인증감율',
                      'thsyr_base_otsd_irds_rt':'금년외지인증감율',
                      'thsyr_base_frnr_irds_rt':'금년외국인증감율'
},inplace=True)
data.head()


,일련번호,시설ID,시설명,위도,경도,전년기준일자,금년기준일자,전년현지인총합,전년외지인총합,전년외국인총합,금년현지인총합,금년외지인총합,금년외국인총합,금년현지인증감수,금년외지인증감수,금년외국인증감수,금년현지인증감율,금년외지인증감율,금년외국인증감율
0,1,KP0001,396커피 컴퍼니,36.560047,128.699928,20190101,20200101,1326,453,0,1445.0,276.0,0.0,119.0,-177.0,0.0,8.97,-39.07,0.0
1,2,KP0001,396커피 컴퍼니,36.560047,128.699928,20190102,20200102,1534,286,0,1562.0,316.0,0.0,28.0,30.0,0.0,1.83,10.49,0.0
2,3,KP0001,396커피 컴퍼니,36.560047,128.699928,20190104,20200104,1571,335,0,1737.0,621.0,0.0,166.0,286.0,0.0,10.57,85.37,0.0
3,4,KP0001,396커피 컴퍼니,36.560047,128.699928,20190106,20200106,1433,445,0,1639.0,299.0,0.0,206.0,-146.0,0.0,14.38,-32.81,0.0
4,5,KP0001,396커피 컴퍼니,36.560047,128.699928,20190107,20200107,1517,347,0,1848.0,319.0,0.0,331.0,-28.0,0.0,21.82,-8.07,0.0


In [ ]:
#금년기준일자와 전년기준일자 년월일로 구분하기위해 문자열로 변경
data['전년기준일자'] = data['전년기준일자'].astype(str)
data['금년기준일자'] = data['금년기준일자'].astype(str)


In [ ]:
#일은 필요없다고 판단 '년도'와 '월'만 사용
data["year1"] =data["전년기준일자"].str[:4]
data["month1"] =data["전년기준일자"].str[4:6]
data["year2"] =data["금년기준일자"].str[:4]
data["month2"] =data["금년기준일자"].str[4:6]
data = data.drop(["전년기준일자","금년기준일자"],axis=1)
data["year1"] = data['year1'].astype(float)
data["month1"] = data['month1'].astype(float)
data["year2"] = data['year2'].astype(float)
data["month2"] = data['month2'].astype(float)
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 992554 entries, 0 to 992553
Data columns (total 21 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   일련번호      992554 non-null  int64  
 1   시설ID      992554 non-null  object 
 2   시설명       992554 non-null  object 
 3   위도        992005 non-null  float64
 4   경도        992005 non-null  float64
 5   전년현지인총합   992554 non-null  int64  
 6   전년외지인총합   992554 non-null  int64  
 7   전년외국인총합   992554 non-null  int64  
 8   금년현지인총합   992554 non-null  float64
 9   금년외지인총합   992554 non-null  float64
 10  금년외국인총합   992554 non-null  float64
 11  금년현지인증감수  992554 non-null  float64
 12  금년외지인증감수  992554 non-null  float64
 13  금년외국인증감수  992554 non-null  float64
 14  금년현지인증감율  992554 non-null  float64
 15  금년외지인증감율  992554 non-null  float64
 16  금년외국인증감율  992554 non-null  float64
 17  year1     992554 non-null  float64
 18  month1    992554 non-null  float64
 19  year2     992554 non-null  float64
 20  mont

In [ ]:
df=data
df.head()

,일련번호,시설ID,시설명,위도,경도,전년현지인총합,전년외지인총합,전년외국인총합,금년현지인총합,금년외지인총합,금년외국인총합,금년현지인증감수,금년외지인증감수,금년외국인증감수,금년현지인증감율,금년외지인증감율,금년외국인증감율,year1,month1,year2,month2
0,1,KP0001,396커피 컴퍼니,36.560047,128.699928,1326,453,0,1445.0,276.0,0.0,119.0,-177.0,0.0,8.97,-39.07,0.0,2019.0,1.0,2020.0,1.0
1,2,KP0001,396커피 컴퍼니,36.560047,128.699928,1534,286,0,1562.0,316.0,0.0,28.0,30.0,0.0,1.83,10.49,0.0,2019.0,1.0,2020.0,1.0
2,3,KP0001,396커피 컴퍼니,36.560047,128.699928,1571,335,0,1737.0,621.0,0.0,166.0,286.0,0.0,10.57,85.37,0.0,2019.0,1.0,2020.0,1.0
3,4,KP0001,396커피 컴퍼니,36.560047,128.699928,1433,445,0,1639.0,299.0,0.0,206.0,-146.0,0.0,14.38,-32.81,0.0,2019.0,1.0,2020.0,1.0
4,5,KP0001,396커피 컴퍼니,36.560047,128.699928,1517,347,0,1848.0,319.0,0.0,331.0,-28.0,0.0,21.82,-8.07,0.0,2019.0,1.0,2020.0,1.0


In [ ]:
#년도와 월별로 그룹화
df.groupby(['시설명','year1','month1','year2','month2'])['전년현지인총합','전년외지인총합','전년외국인총합',
                                     '금년현지인총합','금년외지인총합','금년외국인총합',
                                     '금년현지인증감수','금년외지인증감수','금년외국인증감수',
                                     '금년현지인증감율','금년외지인증감율','금년외국인증감율'].mean()

In [ ]:
#년도와 월별로 그룹화
df.groupby(['시설명','year1','month1'])['전년현지인총합','전년외지인총합','전년외국인총합',
                                     '금년현지인총합','금년외지인총합','금년외국인총합',
                                     '금년현지인증감수','금년외지인증감수','금년외국인증감수',
                                     '금년현지인증감율','금년외지인증감율','금년외국인증감율'].mean()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
#년도와 월별로 그룹화
df1=df.groupby(['시설명','year1','month1'])['위도','경도','전년현지인총합','전년외지인총합','전년외국인총합',
                                     '금년현지인총합','금년외지인총합','금년외국인총합',
                                     '금년현지인증감수','금년외지인증감수','금년외국인증감수',
                                     '금년현지인증감율','금년외지인증감율','금년외국인증감율'].mean()

In [ ]:
df1.to_csv("코로나전후문화시설유동인구_전년월+위도경도.csv" )

## 시설별 증감(현지,외지,외국 비교)
### 지역별 알아보기

In [6]:
#데이터수 992554->21768
df1 = pd.read_csv("코로나전후문화시설유동인구_전년월+위도경도.csv")
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21768 entries, 0 to 21767
Data columns (total 17 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   시설명       21768 non-null  object 
 1   year1     21768 non-null  float64
 2   month1    21768 non-null  float64
 3   위도        21768 non-null  float64
 4   경도        21768 non-null  float64
 5   전년현지인총합   21768 non-null  float64
 6   전년외지인총합   21768 non-null  float64
 7   전년외국인총합   21768 non-null  float64
 8   금년현지인총합   21768 non-null  float64
 9   금년외지인총합   21768 non-null  float64
 10  금년외국인총합   21768 non-null  float64
 11  금년현지인증감수  21768 non-null  float64
 12  금년외지인증감수  21768 non-null  float64
 13  금년외국인증감수  21768 non-null  float64
 14  금년현지인증감율  21768 non-null  float64
 15  금년외지인증감율  21768 non-null  float64
 16  금년외국인증감율  21768 non-null  float64
dtypes: float64(16), object(1)
memory usage: 2.8+ MB


In [8]:
df2=df1.groupby(['시설명'])['위도','경도','전년현지인총합','전년외지인총합','전년외국인총합',
                                     '금년현지인총합','금년외지인총합','금년외국인총합',
                                     '금년현지인증감수','금년외지인증감수','금년외국인증감수',
                                     '금년현지인증감율','금년외지인증감율','금년외국인증감율'].mean()
df2.head()

,위도,경도,전년현지인총합,전년외지인총합,전년외국인총합,금년현지인총합,금년외지인총합,금년외국인총합,금년현지인증감수,금년외지인증감수,금년외국인증감수,금년현지인증감율,금년외지인증감율,금년외국인증감율
시설명,,,,,,,,,,,,,,
(구)송정초등학교 야외운동장,35.182721,129.201629,429.605767,225.084601,1.221928,427.221736,198.509485,0.122453,-2.384031,-26.575115,-1.099475,1.156976,-0.933809,-9.009105
(재)경기문화재단,37.265775,127.036874,418.406612,489.322625,1.247101,533.238582,376.584261,0.260471,114.831970,-112.738364,-0.986630,30.376485,-18.455631,-10.525062
(재)구리시청소년수련관,37.603874,127.141605,5745.991679,2715.243427,3.738511,5505.319387,2246.319566,0.463748,-240.672293,-468.923861,-3.274763,5.620144,-0.257343,-30.517909
(재)영화의전당,35.171082,129.127125,10897.577656,17826.929435,331.872485,6926.269758,9275.790489,22.583173,-3971.307898,-8551.138946,-309.289311,-19.797686,-35.571370,-88.077385
1M SPACE,37.557603,126.938217,869.924603,1728.275147,25.011751,745.864023,1244.828047,2.268369,-124.060580,-483.447101,-22.743382,-9.704160,-21.679918,-55.183980


In [9]:
#시설 수: 1814
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1814 entries, (구)송정초등학교 야외운동장 to 힐크레스트(구. 허브힐즈)
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   위도        1814 non-null   float64
 1   경도        1814 non-null   float64
 2   전년현지인총합   1814 non-null   float64
 3   전년외지인총합   1814 non-null   float64
 4   전년외국인총합   1814 non-null   float64
 5   금년현지인총합   1814 non-null   float64
 6   금년외지인총합   1814 non-null   float64
 7   금년외국인총합   1814 non-null   float64
 8   금년현지인증감수  1814 non-null   float64
 9   금년외지인증감수  1814 non-null   float64
 10  금년외국인증감수  1814 non-null   float64
 11  금년현지인증감율  1814 non-null   float64
 12  금년외지인증감율  1814 non-null   float64
 13  금년외국인증감율  1814 non-null   float64
dtypes: float64(14)
memory usage: 212.6+ KB


In [10]:
df2 = df2.drop(['전년현지인총합','금년현지인총합','금년현지인증감수','금년현지인증감율'],axis = 1)

In [11]:
df2.head()

,위도,경도,전년외지인총합,전년외국인총합,금년외지인총합,금년외국인총합,금년외지인증감수,금년외국인증감수,금년외지인증감율,금년외국인증감율
시설명,,,,,,,,,,
(구)송정초등학교 야외운동장,35.182721,129.201629,225.084601,1.221928,198.509485,0.122453,-26.575115,-1.099475,-0.933809,-9.009105
(재)경기문화재단,37.265775,127.036874,489.322625,1.247101,376.584261,0.260471,-112.738364,-0.986630,-18.455631,-10.525062
(재)구리시청소년수련관,37.603874,127.141605,2715.243427,3.738511,2246.319566,0.463748,-468.923861,-3.274763,-0.257343,-30.517909
(재)영화의전당,35.171082,129.127125,17826.929435,331.872485,9275.790489,22.583173,-8551.138946,-309.289311,-35.571370,-88.077385
1M SPACE,37.557603,126.938217,1728.275147,25.011751,1244.828047,2.268369,-483.447101,-22.743382,-21.679918,-55.183980


In [13]:
df2 = df2.rename_axis('시설명').reset_index()
df2.head()

,시설명,위도,경도,전년외지인총합,전년외국인총합,금년외지인총합,금년외국인총합,금년외지인증감수,금년외국인증감수,금년외지인증감율,금년외국인증감율
0,(구)송정초등학교 야외운동장,35.182721,129.201629,225.084601,1.221928,198.509485,0.122453,-26.575115,-1.099475,-0.933809,-9.009105
1,(재)경기문화재단,37.265775,127.036874,489.322625,1.247101,376.584261,0.260471,-112.738364,-0.986630,-18.455631,-10.525062
2,(재)구리시청소년수련관,37.603874,127.141605,2715.243427,3.738511,2246.319566,0.463748,-468.923861,-3.274763,-0.257343,-30.517909
3,(재)영화의전당,35.171082,129.127125,17826.929435,331.872485,9275.790489,22.583173,-8551.138946,-309.289311,-35.571370,-88.077385
4,1M SPACE,37.557603,126.938217,1728.275147,25.011751,1244.828047,2.268369,-483.447101,-22.743382,-21.679918,-55.183980


In [14]:
df2_rate = df2[['시설명','위도','경도','금년외지인증감율','금년외국인증감율']]
df2_rate['금년외지인외국인증감율'] = (df2_rate['금년외지인증감율'] + df2_rate['금년외국인증감율'])/2
df2_rate = df2_rate.sort_values(by='금년외지인외국인증감율',ascending= False)

In [15]:
df2_rate.head()

,시설명,위도,경도,금년외지인증감율,금년외국인증감율,금년외지인외국인증감율
1371,인천문화예술회관,37.447565,126.700075,64367.236858,82.158744,32224.697801
1352,이화여고 100주년 기념관,37.565828,126.971304,58431.385900,105.555758,29268.470829
1098,안동댐 월영교 일원 개목나루,36.578900,128.764591,27999.617242,6.585006,14003.101124
1427,정동극장,37.565896,126.972796,12387.865318,17.672793,6202.769056
1419,전주소리문화관,35.817804,127.153285,12131.038162,-2.317737,6064.360212


In [19]:
df2_total = df2
df2_total.head()
df2_total['금년외지인외국인증감율'] = (df2_total['금년외지인증감율'] + df2_total['금년외국인증감율'])/2

In [16]:
df2_rate.to_csv("코로나전후문화시설유동인구_증감율.csv" )

In [20]:
df2_total.to_csv("코로나전후문화시설유동인구_외국인외지인final.csv" )

## 총합 top10

In [ ]:
top10_1=df2.sort_values(by='전년현지인총합',ascending= False).head(10)
top10_2=df2.sort_values(by='전년외지인총합',ascending= False).head(10)
top10_3=df2.sort_values(by='전년외국인총합',ascending= False).head(10)
top10_4=df2.sort_values(by='금년현지인총합',ascending= False).head(10)
top10_5=df2.sort_values(by='금년외지인총합',ascending= False).head(10)
top10_6=df2.sort_values(by='금년외국인총합',ascending= False).head(10)


In [ ]:
#전년 현지인 총합 top10
top10_1

,위도,경도,전년현지인총합,전년외지인총합,전년외국인총합,금년현지인총합,금년외지인총합,금년외국인총합,금년현지인증감수,금년외지인증감수,금년외국인증감수,금년현지인증감율,금년외지인증감율,금년외국인증감율
시설명,,,,,,,,,,,,,,
올림픽공원,37.520934,127.122959,51899.919854,40436.976321,468.959927,50316.954463,27850.593807,50.819672,-1582.965392,-12586.382514,-418.140255,-1.782568,-25.774372,-76.482186
서울대학교,37.464007,126.952239,37136.289617,24457.244080,159.247723,32978.107468,16012.533698,24.695811,-4158.182149,-8444.710383,-134.551913,-9.039326,-30.909581,-81.361403
코엑스,37.511662,127.059427,27447.457195,47478.375228,2616.644809,21377.163934,29025.214936,156.952641,-6070.293260,-18453.160291,-2459.692168,-16.349727,-34.019854,-91.760911
서울어린이대공원,37.549899,127.080921,23254.009107,23185.876138,132.424408,24180.282332,15989.120219,29.883424,926.273224,-7196.755920,-102.540984,6.089472,-25.318506,-68.706430
해운대해수욕장,35.157939,129.172315,19961.200364,18451.123862,769.848816,21002.316940,17804.610200,47.400729,1041.116576,-646.513661,-722.448087,5.497923,6.888652,-91.078798
광명시 광명 5동 일대,37.476690,126.849746,18383.684882,9515.302368,36.278689,17490.561020,7514.785064,3.023679,-893.123862,-2000.517304,-33.255009,-4.293607,-18.561075,-88.316922
롯데콘서트홀,37.513110,127.103450,17312.717668,31300.475410,1076.959927,13509.479053,21352.331512,66.109290,-3803.238616,-9948.143898,-1010.850638,-20.545847,-24.084718,-90.204372
벡스코 (BEXCO),35.169097,129.136312,16644.329690,21932.814208,374.635701,15619.551913,15122.479053,13.755920,-1024.777778,-6810.335155,-360.879781,-4.705647,-18.072240,-91.531220
한국과학기술원,36.371779,127.360844,16404.032787,6285.723133,51.732240,13994.488160,4406.930783,11.107468,-2409.544627,-1878.792350,-40.624772,-3.901257,-23.134809,5.685738


In [ ]:
#전년외지인총합 top10
top10_2

,위도,경도,전년현지인총합,전년외지인총합,전년외국인총합,금년현지인총합,금년외지인총합,금년외국인총합,금년현지인증감수,금년외지인증감수,금년외국인증감수,금년현지인증감율,금년외지인증감율,금년외국인증감율
시설명,,,,,,,,,,,,,,
코엑스,37.511662,127.059427,27447.457195,47478.375228,2616.644809,21377.163934,29025.214936,156.952641,-6070.293260,-18453.160291,-2459.692168,-16.349727,-34.019854,-91.760911
올림픽공원,37.520934,127.122959,51899.919854,40436.976321,468.959927,50316.954463,27850.593807,50.819672,-1582.965392,-12586.382514,-418.140255,-1.782568,-25.774372,-76.482186
광화문광장,37.572433,126.976902,6775.149362,31637.914390,1330.411658,6438.570128,18608.051002,84.114754,-336.579235,-13029.863388,-1246.296903,30.854772,-34.191639,-91.382259
롯데콘서트홀,37.513110,127.103450,17312.717668,31300.475410,1076.959927,13509.479053,21352.331512,66.109290,-3803.238616,-9948.143898,-1010.850638,-20.545847,-24.084718,-90.204372
롯데시네마 [월드타워],37.512557,127.102551,15399.972678,29926.679417,1070.757741,11764.468124,20490.593807,64.240437,-3635.504554,-9436.085610,-1006.517304,-21.729654,-23.038087,-90.362386
롯데뮤지엄,37.512557,127.102551,14389.346084,27795.061931,1040.752277,10713.027322,18799.032787,61.664845,-3676.318761,-8996.029144,-979.087432,-23.628670,-23.792732,-90.489545
봉산문화회관,35.863337,128.596556,6793.652095,25469.309654,162.688525,5100.985428,18248.601093,10.916211,-1692.666667,-7220.708561,-151.772313,-23.776339,-21.090109,-86.051767
영등포 타임스퀘어,37.517233,126.903716,10956.220401,25134.706740,338.298725,10330.805100,18666.358834,25.856102,-625.415301,-6468.347905,-312.442623,-2.976776,-15.746175,-89.896375
CGV신한카드아트홀,37.517233,126.903716,10956.220401,25134.706740,338.298725,10330.805100,18666.358834,25.856102,-625.415301,-6468.347905,-312.442623,-2.976776,-15.746175,-89.896375


In [ ]:
#전년 외국인 총합 top10
top10_3

,위도,경도,전년현지인총합,전년외지인총합,전년외국인총합,금년현지인총합,금년외지인총합,금년외국인총합,금년현지인증감수,금년외지인증감수,금년외국인증감수,금년현지인증감율,금년외지인증감율,금년외국인증감율
시설명,,,,,,,,,,,,,,
롯데호텔 [서울],37.565427,126.981621,3431.438980,15491.437158,5253.253188,3152.156648,11866.704918,299.657559,-279.282332,-3624.732240,-4953.595628,2.511603,-19.817614,-92.987450
국립극단 [명동],37.564046,126.984220,1534.717668,8525.795993,3611.081967,1224.590164,5419.806922,239.455373,-310.127505,-3105.989071,-3371.626594,-10.128707,-33.802386,-91.742969
코엑스,37.511662,127.059427,27447.457195,47478.375228,2616.644809,21377.163934,29025.214936,156.952641,-6070.293260,-18453.160291,-2459.692168,-16.349727,-34.019854,-91.760911
영락교회,37.563185,126.989196,1725.353370,7022.078324,2336.821494,1418.961749,3979.384335,144.149362,-306.391621,-3042.693989,-2192.672131,-4.416758,-38.009781,-91.730273
명동대성당,37.563335,126.987274,1243.610200,7176.326047,2114.346084,1020.531876,3962.499089,127.601093,-223.078324,-3213.826958,-1986.744991,-10.657960,-40.186230,-92.016794
삼일로창고극장,37.563481,126.988017,1243.610200,7176.326047,2114.346084,1020.531876,3962.499089,127.601093,-223.078324,-3213.826958,-1986.744991,-10.657960,-40.186230,-92.016794
정태호소극장 comedy maker,37.553138,126.922212,2865.706740,13483.409836,2079.378871,1836.921676,6596.402550,126.437158,-1028.785064,-6887.007286,-1952.941712,-34.308597,-42.145246,-91.675829
블루라이트 라이브 홀,37.553138,126.922212,2865.706740,13483.409836,2079.378871,1836.921676,6596.402550,126.437158,-1028.785064,-6887.007286,-1952.941712,-34.308597,-42.145246,-91.675829
아이디어스 크래프트하우스,37.553620,126.920935,2032.300546,9219.351548,2042.160291,1029.000000,2180.018215,119.595628,-1003.300546,-7039.333333,-1922.564663,-48.497268,-74.546339,-91.567614


In [ ]:
#금년 현지인 총합 top10
top10_4

,위도,경도,전년현지인총합,전년외지인총합,전년외국인총합,금년현지인총합,금년외지인총합,금년외국인총합,금년현지인증감수,금년외지인증감수,금년외국인증감수,금년현지인증감율,금년외지인증감율,금년외국인증감율
시설명,,,,,,,,,,,,,,
올림픽공원,37.520934,127.122959,51899.919854,40436.976321,468.959927,50316.954463,27850.593807,50.819672,-1582.965392,-12586.382514,-418.140255,-1.782568,-25.774372,-76.482186
서울대학교,37.464007,126.952239,37136.289617,24457.244080,159.247723,32978.107468,16012.533698,24.695811,-4158.182149,-8444.710383,-134.551913,-9.039326,-30.909581,-81.361403
서울어린이대공원,37.549899,127.080921,23254.009107,23185.876138,132.424408,24180.282332,15989.120219,29.883424,926.273224,-7196.755920,-102.540984,6.089472,-25.318506,-68.706430
코엑스,37.511662,127.059427,27447.457195,47478.375228,2616.644809,21377.163934,29025.214936,156.952641,-6070.293260,-18453.160291,-2459.692168,-16.349727,-34.019854,-91.760911
해운대해수욕장,35.157939,129.172315,19961.200364,18451.123862,769.848816,21002.316940,17804.610200,47.400729,1041.116576,-646.513661,-722.448087,5.497923,6.888652,-91.078798
신세계 문화홀 [의정부],37.737990,127.045879,16269.446266,12843.213115,41.358834,18247.723133,10123.513661,4.353370,1978.276867,-2719.699454,-37.005464,16.120000,-10.090838,-78.924791
광명시 광명 5동 일대,37.476690,126.849746,18383.684882,9515.302368,36.278689,17490.561020,7514.785064,3.023679,-893.123862,-2000.517304,-33.255009,-4.293607,-18.561075,-88.316922
벡스코 (BEXCO),35.169097,129.136312,16644.329690,21932.814208,374.635701,15619.551913,15122.479053,13.755920,-1024.777778,-6810.335155,-360.879781,-4.705647,-18.072240,-91.531220
경주 황성공원 일원,35.859489,129.213983,14287.329690,2868.684882,15.921676,14606.967213,2504.418944,0.604736,319.637523,-364.265938,-15.316940,5.262750,1.158160,-40.009982


In [ ]:
##금년외지인총합 top10
top10_5

,위도,경도,전년현지인총합,전년외지인총합,전년외국인총합,금년현지인총합,금년외지인총합,금년외국인총합,금년현지인증감수,금년외지인증감수,금년외국인증감수,금년현지인증감율,금년외지인증감율,금년외국인증감율
시설명,,,,,,,,,,,,,,
코엑스,37.511662,127.059427,27447.457195,47478.375228,2616.644809,21377.163934,29025.214936,156.952641,-6070.293260,-18453.160291,-2459.692168,-16.349727,-34.019854,-91.760911
올림픽공원,37.520934,127.122959,51899.919854,40436.976321,468.959927,50316.954463,27850.593807,50.819672,-1582.965392,-12586.382514,-418.140255,-1.782568,-25.774372,-76.482186
롯데콘서트홀,37.513110,127.103450,17312.717668,31300.475410,1076.959927,13509.479053,21352.331512,66.109290,-3803.238616,-9948.143898,-1010.850638,-20.545847,-24.084718,-90.204372
롯데시네마 [월드타워],37.512557,127.102551,15399.972678,29926.679417,1070.757741,11764.468124,20490.593807,64.240437,-3635.504554,-9436.085610,-1006.517304,-21.729654,-23.038087,-90.362386
롯데뮤지엄,37.512557,127.102551,14389.346084,27795.061931,1040.752277,10713.027322,18799.032787,61.664845,-3676.318761,-8996.029144,-979.087432,-23.628670,-23.792732,-90.489545
CGV신한카드아트홀,37.517233,126.903716,10956.220401,25134.706740,338.298725,10330.805100,18666.358834,25.856102,-625.415301,-6468.347905,-312.442623,-2.976776,-15.746175,-89.896375
영등포 타임스퀘어,37.517233,126.903716,10956.220401,25134.706740,338.298725,10330.805100,18666.358834,25.856102,-625.415301,-6468.347905,-312.442623,-2.976776,-15.746175,-89.896375
광화문광장,37.572433,126.976902,6775.149362,31637.914390,1330.411658,6438.570128,18608.051002,84.114754,-336.579235,-13029.863388,-1246.296903,30.854772,-34.191639,-91.382259
봉산문화회관,35.863337,128.596556,6793.652095,25469.309654,162.688525,5100.985428,18248.601093,10.916211,-1692.666667,-7220.708561,-151.772313,-23.776339,-21.090109,-86.051767


In [ ]:
#금년외국인 총합 top10
top10_6

,위도,경도,전년현지인총합,전년외지인총합,전년외국인총합,금년현지인총합,금년외지인총합,금년외국인총합,금년현지인증감수,금년외지인증감수,금년외국인증감수,금년현지인증감율,금년외지인증감율,금년외국인증감율
시설명,,,,,,,,,,,,,,
롯데호텔 [서울],37.565427,126.981621,3431.438980,15491.437158,5253.253188,3152.156648,11866.704918,299.657559,-279.282332,-3624.732240,-4953.595628,2.511603,-19.817614,-92.987450
국립극단 [명동],37.564046,126.984220,1534.717668,8525.795993,3611.081967,1224.590164,5419.806922,239.455373,-310.127505,-3105.989071,-3371.626594,-10.128707,-33.802386,-91.742969
코엑스,37.511662,127.059427,27447.457195,47478.375228,2616.644809,21377.163934,29025.214936,156.952641,-6070.293260,-18453.160291,-2459.692168,-16.349727,-34.019854,-91.760911
영락교회,37.563185,126.989196,1725.353370,7022.078324,2336.821494,1418.961749,3979.384335,144.149362,-306.391621,-3042.693989,-2192.672131,-4.416758,-38.009781,-91.730273
MHN STAGE,37.556098,126.923458,2780.969035,10948.193078,1979.746812,2270.606557,6582.451730,134.956284,-510.362477,-4365.741348,-1844.790528,-15.355883,-32.100984,-90.142532
명동대성당,37.563335,126.987274,1243.610200,7176.326047,2114.346084,1020.531876,3962.499089,127.601093,-223.078324,-3213.826958,-1986.744991,-10.657960,-40.186230,-92.016794
삼일로창고극장,37.563481,126.988017,1243.610200,7176.326047,2114.346084,1020.531876,3962.499089,127.601093,-223.078324,-3213.826958,-1986.744991,-10.657960,-40.186230,-92.016794
블루라이트 라이브 홀,37.553138,126.922212,2865.706740,13483.409836,2079.378871,1836.921676,6596.402550,126.437158,-1028.785064,-6887.007286,-1952.941712,-34.308597,-42.145246,-91.675829
정태호소극장 comedy maker,37.553138,126.922212,2865.706740,13483.409836,2079.378871,1836.921676,6596.402550,126.437158,-1028.785064,-6887.007286,-1952.941712,-34.308597,-42.145246,-91.675829


## 월별 증감(현지,외지,외국)